In [57]:
%run FormattedParam.ipynb

In [58]:
import param as pm
import panel as pn
import pandas as pd
import millify
import numbers
import hvplot.pandas
pn.extension()

In [113]:
class Forest(FormattedParam):
    forest_name = pm.String("KNF1")
    forested_area_hectares = pm.Number(7478, precedence=1)
    total_centree_count = pm.Number(101675, precedence=1)
    tons_carbon_per_year = pm.Number(21867.58, precedence=1)
    stewardship_rate_per_hectare = pm.Number(28, precedence=-1)
    
    extractive_tenure_purchase = pm.Number(1884615.38, precedence=1)
    legal_fees_convert_tenure_to_living_tenure = pm.Number(175000, precedence=-1)
    due_diligence_data_audit_dollar_per_hectare = pm.Number(8, precedence=-1)
    
    insurance_pool_rate = pm.Number(0.1, bounds=(0,1), step=0.01, precedence=-1)
    
    centree_minting_fee = pm.Number(0.07, bounds=(0,1), step=0.01, precedence=-1)
    
    sale_value_factor = pm.Number(0.2, bounds=(0,1), step=0.01, precedence=-1)
    
    centree_commission_rate = pm.Number(0.4, bounds=(0,1), step=0.01, precedence=-1)
    
    method_fields = [
            'acres_protected', 'due_dilligence_cost', 'subtotal', 'insurance_pool', 'total_protection_cost', 
            'centree_count_per_hectare', 'protection_cost_per_hectare', 'centree_mint_fee_per_hectare',
            'total_minting_charge', 'total_cost_per_hectare', 'total_cost_per_centree', 'total_production_cost_including_minting',
            'sale_value', 'cost_per_hectare', 'cost_per_acre', 'centree_price', 'gross_return', 'centree_commission',
            'landowner_share', 'centree_commission_per_hectare', 'centree_commission_of_sale', 'landowner_share_of_sale',
            'total_centree_revenue', 'total_centree_revenue_per_asset_value',
        ]
    
    def acres_protected(self):
        return self.forested_area_hectares * 2.47
    
    def due_dilligence_cost(self):
        return self.due_diligence_data_audit_dollar_per_hectare * self.forested_area_hectares
    
    def subtotal(self):
        return self.extractive_tenure_purchase + self.legal_fees_convert_tenure_to_living_tenure + self.due_dilligence_cost()
    
    def insurance_pool(self):
        return self.subtotal() * self.insurance_pool_rate
    
    def total_protection_cost(self):
        return self.subtotal() + self.insurance_pool()
    
    def centree_count_per_hectare(self):
        return self.total_centree_count / self.forested_area_hectares
    
    def protection_cost_per_hectare(self):
        return self.total_protection_cost() / self.forested_area_hectares
    
    def centree_mint_fee_per_hectare(self):
        return self.protection_cost_per_hectare() * self.centree_minting_fee
    
    def total_minting_charge(self):
        return self.centree_mint_fee_per_hectare() * self.forested_area_hectares
    
    def total_cost_per_hectare(self):
        return self.protection_cost_per_hectare() + self.centree_mint_fee_per_hectare()
    
    def total_cost_per_centree(self):
        return self.total_cost_per_hectare() / self.centree_count_per_hectare()
    
    def total_production_cost_including_minting(self):
        return self.total_cost_per_hectare() * self.forested_area_hectares
    
    def sale_value(self):
        return self.total_production_cost_including_minting() / (1 - self.sale_value_factor)
    
    def cost_per_hectare(self):
        return self.sale_value() / self.forested_area_hectares
    
    def cost_per_acre(self):
        return self.cost_per_hectare() / 2.7
    
    def centree_price(self):
        return self.sale_value() / self.total_centree_count
    
    def gross_return(self):
        return self.sale_value() - self.total_production_cost_including_minting()
    
    def centree_commission(self):
        return self.gross_return() * self.centree_commission_rate
    
    def landowner_share(self):
        return self.gross_return() - self.centree_commission()
    
    def centree_commission_per_hectare(self):
        return self.centree_commission() / self.forested_area_hectares
    
    def centree_commission_of_sale(self):
        return self.centree_commission() / self.sale_value()
    
    def landowner_share_of_sale(self):
        return self.landowner_share() / self.sale_value()
    
    def total_centree_revenue(self):
        return self.total_minting_charge() + self.centree_commission()
    
    def total_centree_revenue_per_asset_value(self):
        return self.total_centree_revenue() / self.sale_value()
    
    def param_table(self):
        param_info  = [(k,v,'param') for k,v in self.param.values().items()]
        param_table = pd.DataFrame(param_info, columns=['Field', 'Value', 'Type'])
        return param_table
    
    def method_table(self):
        method_info = [(field, getattr(self, field)(),'method') for field in self.method_fields]
        method_table = pd.DataFrame(method_info, columns=['Field', 'Value', 'Type'])
        return method_table
    
    def info_table(self):
        table = pd.concat([self.param_table(), self.method_table()])
        return table
    
    @staticmethod
    def _format_numeric(v):
        return millify.prettify(round(v,2)) if isinstance(v, numbers.Number) else v
    
    def view_info_table(self):
        table = self.info_table()
        table['Value'] = table['Value'].apply(self._format_numeric)
        return table.hvplot.table(height=1000)
    
    def export(self):
        table = self.info_table()
        return table.pivot(columns='Field', values='Value').bfill().iloc[:1].set_index('forest_name')

In [114]:
f1 = Forest(
    forest_name="KNF1",
    forested_area_hectares=7478, 
    total_centree_count=101675,
    tons_carbon_per_year=21867.58,
    extractive_tenure_purchase = 1884615.38,
    legal_fees_convert_tenure_to_living_tenure = 175000,
    due_diligence_data_audit_dollar_per_hectare = 8,
)

In [115]:
pn.Row(f1.view, f1.view_info_table)

Row
    [0] ParamMethod(method, _pane=Param)
    [1] ParamMethod(method, _pane=HoloViews)

In [116]:
f2 = Forest(
    forest_name="KNF2",
    forested_area_hectares=15371, 
    total_centree_count=785944,
    tons_carbon_per_year= 40045.9, # (21867.58 * (15371/7478))
    extractive_tenure_purchase = 3873819.60,
    legal_fees_convert_tenure_to_living_tenure = 175000,
    due_diligence_data_audit_dollar_per_hectare = 8,
)

In [117]:
pn.Row(f2.view, f2.view_info_table)

Row
    [0] ParamMethod(method, _pane=Param)
    [1] ParamMethod(method, _pane=HoloViews)

In [118]:
forests = [f1, f2]

In [133]:
df = pd.concat([f.export() for f in forests]).T.applymap(Forest._format_numeric)

In [135]:
df

forest_name,KNF1,KNF2
Field,,
acres_protected,"18,470.66","37,966.37"
centree_commission,"249,458.02","491,019.4"
centree_commission_of_sale,0.08,0.08
centree_commission_per_hectare,33.36,31.94
centree_commission_rate,0.4,0.4
centree_count_per_hectare,13.6,51.13
centree_mint_fee_per_hectare,21.82,20.9
centree_minting_fee,0.07,0.07
centree_price,30.67,7.81
